In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from google.colab.patches import cv2_imshow
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import VGG19

import dlib, cv2, os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.patheffects as path_effects
import numpy as np
import pandas as pd
import tensorflow as tf

# **표정 분석 모델링**

## **1. 데이터 불러오기**

In [ ]:
# 이미지 데이터 로드
import pickle
import gzip

with gzip.open('C:/Users/jisan/workspace/표정 감정 분석/Emotion_Detection/Data_Preprocessing/dataset',"rb") as fr:
    img_result = pickle.load(fr)
with gzip.open('C:/Users/jisan/workspace/표정 감정 분석/Emotion_Detection/Data_Preprocessing/dataset',"rb") as fr:
    emotion_result = pickle.load(fr)
with gzip.open('C:/Users/jisan/workspace/표정 감정 분석/Emotion_Detection/Data_Preprocessing/dataset',"rb") as fr:
    face_result = pickle.load(fr)

In [ ]:
# 이미지와 감정 라벨의 길이가 같은 것을 확인
len(img_result), len(emotion_result), len(face_result)

In [ ]:
# Face_result 결과 확인
fig, axes = plt.subplots(1, 2, figsize=(16,10))
axes[0].imshow(cv2.cvtColor(img_result[100], cv2.COLOR_BGR2RGB))
axes[1].imshow(cv2.cvtColor(face_result[100], cv2.COLOR_BGR2RGB))

## **2. 데이터 행렬 처리**

In [ ]:
# 데이터 list reshape
image_set = np.asarray(face_result)
emotion_name = np.asarray(emotion_result)
# image_set(x값)과 emotion_name(y값)의 shape 확인
image_set.shape, emotion_name.shape

In [ ]:
# Train_set, Test_set 만들기
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# x값 형태 맞추기
X = image_set[:, :, :, :]

# String 형태의 y값을 LabelEncoder로 변환
Y_obj = emotion_name[:]
e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)

# Train, Tes set형성 / stratify를 통해 y 기준으로 셔플
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, stratify=Y)

# Y_train과 Y_test의 형태 변경
Y_train = Y_train.reshape(-1, 1)
Y_test = Y_test.reshape(-1,1)

# X_train, Y_train 형태 확인
X_train.shape, Y_train.shape

## **3. 모델링**

In [ ]:
# CNN
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', strides=(2,2), padding='SAME', input_shape=(46, 46, 3)))
model.add(layers.MaxPooling2D((2, 2), padding='SAME'))
#model.add(layers.Dropout(0.5))

model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='SAME', strides=(2,2)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='SAME', strides=(2,2)))
model.add(layers.MaxPooling2D((2, 2), padding='SAME'))
model.add(layers.Dropout(0.5))

model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='SAME', strides=(2,2)))
model.add(layers.MaxPooling2D((2, 2), padding='SAME'))
model.add(layers.Dropout(0.5))

model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='SAME', strides=(2,2)))
model.add(layers.MaxPooling2D((2, 2), padding='SAME'))
model.add(layers.Dropout(0.5))

model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='SAME', strides=(2,2)))
model.add(layers.MaxPooling2D((2, 2), padding='SAME'))
model.add(layers.Dropout(0.5))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))

In [ ]:
# 16층 VGG넷 모델
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu',input_shape=(256, 256, 3)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='SAME'))
model.add(layers.MaxPooling2D((2, 2), padding='SAME'))

model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='SAME'))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='SAME'))
model.add(layers.MaxPooling2D((2, 2), padding='SAME'))

model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='SAME'))
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='SAME'))
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='SAME'))
model.add(layers.MaxPooling2D((2, 2), padding='SAME'))

model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='SAME'))
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='SAME'))
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='SAME'))
model.add(layers.MaxPooling2D((2, 2), padding='SAME'))

model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='SAME'))
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='SAME'))
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='SAME'))
model.add(layers.MaxPooling2D((2, 2), padding='SAME'))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))

In [ ]:
# 13층 VGG넷 모델
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu',input_shape=(256, 256, 3)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='SAME'))
model.add(layers.MaxPooling2D((2, 2), padding='SAME'))
model.add(layers.Dropout(0.5))

model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='SAME'))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='SAME'))
model.add(layers.MaxPooling2D((2, 2), padding='SAME'))
model.add(layers.Dropout(0.5))

model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='SAME'))
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='SAME'))
model.add(layers.MaxPooling2D((2, 2), padding='SAME'))
model.add(layers.Dropout(0.5))

model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='SAME'))
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='SAME'))
model.add(layers.MaxPooling2D((2, 2), padding='SAME'))
model.add(layers.Dropout(0.5))

model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='SAME'))
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='SAME'))
model.add(layers.MaxPooling2D((2, 2), padding='SAME'))
model.add(layers.Dropout(0.5))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))

In [ ]:
# ALEX넷 모델
model = models.Sequential()

# 1번째 Convolution Layer
model.add(layers.Conv2D(64, (3, 3), activation='relu',input_shape=(256, 256, 3)))
model.add(layers.MaxPooling2D((2, 2), padding='SAME'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.8))

# 2번째 Convolution Layer
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='SAME'))
model.add(layers.MaxPooling2D((2, 2), padding='SAME'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.8))

# 3번째 Convolution Layer
model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='SAME'))
model.add(layers.MaxPooling2D((2, 2), padding='SAME'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.8))

# 4번째 Convolution Layer
model.add(layers.Conv2D(512, (3, 3), activation='relu', padding='SAME'))
model.add(layers.MaxPooling2D((2, 2), padding='SAME'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.8))

# 5번째 Convolution Layer
model.add(layers.Conv2D(1024, (3, 3), activation='relu', padding='SAME'))
model.add(layers.MaxPooling2D((2, 2), padding='SAME'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.8))

# Fully Connected Layer
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(7, activation='softmax'))# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Resnet50
from keras import models, layers
from keras import Input
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add
 
# number of classes
K = 7

input_tensor = Input(shape=(256, 256, 3), dtype='float32', name='input')
 
def conv1_layer(x):    
    x = ZeroPadding2D(padding=(3, 3))(x)
    x = Conv2D(64, (7, 7), strides=(2, 2))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(padding=(1,1))(x)
    return x  
 
def conv2_layer(x):         
    x = MaxPooling2D((3, 3), 2)(x)     
    shortcut = x

    for i in range(3):
        if (i == 0):
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(shortcut)            
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)
 
            x = Add()([x, shortcut])
            x = Activation('relu')(x)
            shortcut = x
 
        else:
            x = Conv2D(64, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(64, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])   
            x = Activation('relu')(x)  
 
            shortcut = x          
    return x
 
def conv3_layer(x):        
    shortcut = x
    
    for i in range(4):     
        if(i == 0):            
            x = Conv2D(128, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)    
 
            shortcut = x              
        
        else:
            x = Conv2D(128, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(128, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])     
            x = Activation('relu')(x)
 
            shortcut = x      
            
    return x
 
 
 
def conv4_layer(x):
    shortcut = x        
  
    for i in range(6):     
        if(i == 0):            
            x = Conv2D(256, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(1024, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)
 
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)
 
            shortcut = x               
        
        else:
            x = Conv2D(256, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(256, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(1024, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)            
 
            x = Add()([x, shortcut])    
            x = Activation('relu')(x)
 
            shortcut = x      
    return x
 
def conv5_layer(x):
    shortcut = x    
  
    for i in range(3):     
        if(i == 0):            
            x = Conv2D(512, (1, 1), strides=(2, 2), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)        
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)  
 
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
            shortcut = Conv2D(2048, (1, 1), strides=(2, 2), padding='valid')(shortcut)
            x = BatchNormalization()(x)
            shortcut = BatchNormalization()(shortcut)            
 
            x = Add()([x, shortcut])  
            x = Activation('relu')(x)      
 
            shortcut = x               
        
        else:
            x = Conv2D(512, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            
            x = Conv2D(512, (3, 3), strides=(1, 1), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
 
            x = Conv2D(2048, (1, 1), strides=(1, 1), padding='valid')(x)
            x = BatchNormalization()(x)           
            
            x = Add()([x, shortcut]) 
            x = Activation('relu')(x)       
 
            shortcut = x                  
 
    return x
 
x = conv1_layer(input_tensor)
x = conv2_layer(x)
x = conv3_layer(x)
x = conv4_layer(x)
#x = conv5_layer(x)
 
x = GlobalAveragePooling2D()(x)
output_tensor = Dense(K, activation='softmax')(x)
 
model = Model(input_tensor, output_tensor)

In [ ]:
# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
# 모델 피팅
model.fit(X_train, Y_train, epochs = 100, validation_split=0.2)

In [ ]:
final_model = models.load_model('/content/models/Resnet40.h5')

In [ ]:
test_loss, test_acc = model.evaluate(X_test, Y_test, verbose=1)
print(test_acc)

## **3. 모델 테스트**

In [ ]:
label_name = ['분노', '혐오', '공포', '기쁨', '중립', '슬픔', '놀람']

In [ ]:
# 테스트 이미지 파일 디렉토리 연결
testing_images = '/content/test_data'

# 이미지 파일, 이모션 라벨 리스트
test_list = []

# 테스트 이미지 리스트에 담기
for i in os.listdir(testing_images):
    if 'png' or 'tiff' or 'jpg' in i:
        test_list.append(testing_images + '/' + i)
    else:
        pass

test_list

In [ ]:
# 빈 리스트 생성
test_result = []
test_face = []

# 얼굴 디렉터 모듈 초기화
detector = dlib.get_frontal_face_detector()
# object 디텍션 모듈 초기화
objs = dlib.full_object_detections()
# 얼굴 특징점 모듈 초기화
predictor = dlib.shape_predictor('/content/models/shape_predictor_5_face_landmarks.dat')

# align_faces 함수 정의
def align_faces(img):
    dets = detector(img, 1)
    objs = dlib.full_object_detections()
    for detection in dets:
        s = predictor(img, detection)
        objs.append(s)
    faces = dlib.get_face_chips(img, objs, size=256, padding=0.35)
    return faces

for i, img in enumerate(test_list[1:3]):
    # img를 image 형태로 변경
    img = cv2.imread(img)
    h, w = img.shape[:2]
    # 얼굴 인식
    faces = detector(img)
    # 얼굴이 인식되지 않을 경우
    if len(faces) == 0:
        pass
    
    # 얼굴이 인식될 경우
    else:
        # 얼굴이 인식된 원본 사진 저장
        test_result.append(img)
        # 얼굴이 인식된 얼굴 사진 저장
        test_face.append(align_faces(img))

In [ ]:
# 이미지 형태 확인
check_test = np.array(test_face)
check_test = check_test[:, 0, :, :]
check_test.shape

In [ ]:
# test 이미지의 예측 결과
preds = model.predict(check_test)
#print(preds)

# 예측 결과 출력
for i, j in enumerate(preds):
    # 사진 출력
    img = test_result[i]
    cv2_imshow(cv2.resize(img, (256, 256), interpolation=True))
    # 결과물 출력
    print("사진의 표정은 {:.4f}%로 {}입니다".format(max(j)*100,label_name[np.argmax(j)]))